In [64]:
import subprocess
import json
from openai import OpenAI

In [3]:
with open('../config/config.json') as f:
     config = json.load(f)

client = OpenAI(api_key=config['OPENAI_API_KEY'])

In [32]:
design_prompt = f"""
You are an expert in mechanical engineering. 
You are familar with different mechanical components and system running mechanism. 
You task is to design and compute a set of parameters based on user's requirement. 
You final analysis reuslt should be written in markdown syntax for text formalization.
Please analyze the process step by step.
"""

In [33]:
design_requirement = f""" 
Suppose there is a one-stage system with two spur gears,
their gear axes are parallel and the distance between two axes is 100mm. 
Currently, there is a requirements for the reduction system that the desired gear ratio is 3:1, 
Set the driving gear to the center of the plane, which is the coordinate (0, 0)
Based on the given information, computer module number, teeth number of both gears and the coordniate of the driven gear centers.
Please give the result in the format given below:

EXAMPLE: 
gear 1 (driving gear): spur gear, module 2.5mm, 20 teeth, pitch diameter 50 mm, coordinates (0,0)
gear 2 (driven gear): spur gear, module 2.5mm, 60 teeth, pitch diameter 150 mm, coordinates (100,0)
"""

In [34]:
messages=[
        {"role": "system", "content": design_prompt},
        {"role": "user", "content": design_requirement},
    ]
stop_tokens = ['']
collected_tokens = []

res = client.chat.completions.create(
    model=config['GPT_MODEL'],
    messages=messages,
    stream=True,
)

for chunk in res:
    temp_tokens = chunk.choices[0].delta.content
    if temp_tokens not in stop_tokens:
        collected_tokens.append(temp_tokens)
    else:
        chunk_res = "".join(collected_tokens).strip()
        print(chunk_res)
        collected_tokens = []


The gear ratio indicates the relationship between the speeds, torques, and diameters of two gears in mesh
For a gear ratio of 3:1, the driven gear must have three times as many teeth as the driving gear, or the driving gear must have three times as small a diameter compared to the driven gear.

Here's how we can compute the parameters for the gears:

1
**Choose a standard module size**: The module of a gear is a measure of the gear tooth size which is equal to the pitch diameter divided by the number of teeth
The module (m) must be chosen such that the gears will have a whole number of teeth
The module size will also depend on factors like the power transmission requirements and space constraints
For simplicity, we can start with an arbitrary module size and adjust later if needed.

2
**Calculate teeth number for both gears**: Given the 3:1 gear ratio, if we assign the driving gear (smaller gear) Z1 teeth, the driven gear (larger gear) will have Z2 = 3 * Z1 teeth.

3
**Pitch diameters

In [17]:
parsing_prompt = f"""
You are an expert in commanding and instructions construction in JSON files. 
You receive a textural description of a set of parameters for external spur gear system designing.
You should analysis and detect parameters writen in the textual description accurately.
You are required to parse a json file in the provided format based on the description for formalization.
You can fill the provided json format by replacing '' in each item with parameters in the given description.
For all coordinate parameters in the json file should be represented in numerical value, not in string format.
For rotation parameters in coordinate: alpha, bete, and gamma are all set to 0 in default 
"""

parsing_format = {
    "gear 1":{
        "gear_type": '',
        "coordinate": {"x": '', "y": '', "z": '', 'alpha': '', 'beta': '', 'gamma': ''},
        "unit": '',
        "module": '',
        "teeth": '',
        "pitch_d": ''
    },
    "gear 2": {
        "gear_type": '',
        "coordinate": {"x": '', "y": '', "z": '', 'alpha': '', 'beta': '', 'gamma': ''},
        "unit": '',
        "module": '',
        "teeth": '',
        "pitch_d": ''
    }
}

parsing_prompt = parsing_prompt + str(parsing_format)

In [30]:
parsing_description = f"""
gear 1 (driving gear): spur gear, module 2.5mm, 20 teeth, pitch diameter 50 mm, coordinates (0,0)
gear 2 (driven gear): spur gear, module 2.5mm, 60 teeth, pitch diameter 150 mm, coordinates (100,0)
"""
                    
parsing_messages = [
    {"role": "system", "content": parsing_prompt},
    {"role": "user", "content": parsing_description},
]

In [19]:
parsing_response = client.chat.completions.create(
    model=config['GPT_MODEL'],
    response_format={ "type": "json_object" },
    messages=parsing_messages,
)

In [20]:
gears_parameters = parsing_response.choices[0].message.content.strip()
print(gears_parameters)

{
    "gear 1": {
        "gear_type": "spur",
        "coordinate": {
            "x": 0,
            "y": 0,
            "z": 0,
            "alpha": 0,
            "beta": 0,
            "gamma": 0
        },
        "unit": "mm",
        "module": 2.5,
        "teeth": 20,
        "pitch_d": 50
    },
    "gear 2": {
        "gear_type": "spur",
        "coordinate": {
            "x": 100,
            "y": 0,
            "z": 0,
            "alpha": 0,
            "beta": 0,
            "gamma": 0
        },
        "unit": "mm",
        "module": 2.5,
        "teeth": 60,
        "pitch_d": 150
    }
}


In [21]:
with open("../common/spur_gear/gears_parameter.json", "w") as f:
    f.write(gears_parameters)

In [53]:
model_introdcution = f"""
You are an expert in 3D modeling with Computer-aided design package.
You are currently required to used OpenSCAD package to build the model based on a given design parameters in Json format.
You need to write a model script in OpenSCAD to parametrically build the 3D geometry with parameters in the given json file. 
You should analyze each item in the Json file carefully to extract all parameters including:"""

json_format = {
    "gear 1": {
        "gear_type": "spur",
        "coordinate": {"x": 0, "y": 0, "z": 0, "alpha": 0, "beta": 0, "gamma": 0},
        "unit": "mm",
        "module": 2.5,
        "teeth": 20,
        "pitch_d": 50
    },
}

model_script = f"""
REMEMBER to follow all mentioned formats strictlly.

Firstly, you have to move the gear into a correct position based on the parameters via a sequence of functions in the format:
translate([x, y, z]) rotate([alpha, beta gamma])

Then based on these information, you should first choose correct function based on the value in "gear type":
spur gear => gear();  
helix gear => gear_helix(); 
herringbone gear => gear_herringbone(); 
bevel gear => gear_bevel(); 

After that, you have to put the "module" and "teeth" to the choosen function in the format:
func(m = modulus, z = teeth, x = 0, w = 20, h = 4)

Thus you can get a final reuslt for one item in the script:
translate([x, y, z]) rotate([alpha, beta gamma]) func(m = modulus, z = teeth, x = 0, w = 20, h = 4);

Keep doing this until you finish all items in the Json file without other textual description. 
NO other natural language description or symbols are needed.
"""

scripting_prompt = model_introdcution + str(json_format) + model_script

In [54]:
with open("../common/spur_gear/gears_parameter.json", "r") as f:
    parameters = json.load(f)
parameters = str(parameters)

scripting_description = f"Write a script according to provided procedure with parameters of gears" + parameters

In [55]:
script_messages = [
    {"role": "system", "content": scripting_prompt},
    {"role": "user", "content": scripting_description}
]

In [56]:
scripting_response = client.chat.completions.create(
    model=config['GPT_MODEL'],
    messages=script_messages,
)

In [57]:
script = scripting_response.choices[0].message.content.strip()
print(script)

translate([0, 0, 0]) rotate([0, 0, 0]) gear(m = 2.5, z = 20);
translate([100, 0, 0]) rotate([0, 0, 0]) gear(m = 2.5, z = 60);


In [60]:
with open("../common/spur_gear/gears_script.scad", "w") as f:
    f.write("include <gears.scad>\n")

with open("../common/spur_gear/gears_script.scad", "a") as f:
    f.write("\n" + script)

In [66]:
subprocess.Popen(["D:\openSCAD\install\openscad.exe",
                 "../common/spur_gear/gears_script.scad"])

<Popen: returncode: None args: ['D:\\openSCAD\\install\\openscad.exe', '../c...>